In [1]:
# to handle datasets
import pandas as pd
from pandas import DataFrame
pd.pandas.set_option('display.max_columns', None)
import numpy as np

# to plot
import matplotlib.pyplot as plt
import seaborn as sns

# divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

In [2]:
# load dataset and drop the unneeded columns in the dataFrame
# rows with no gametime played, unnamed: 0 and datetime
ffmlDf = pd.read_csv('ffmlDF_20-21')

ffmlDf.head()

,Unnamed: 0,points,minsPlayed,goalsScored,assists,cleanSheets,goalsConceded,ownGoals,penSaved,penMissed,yelCards,redCards,saves,bonus,bonusPointSystem,influence,creativity,threat,ictIndex,netTransfers,selectedBy,costGBP,gameDate,playerName,oppositionTeam
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,76656,7.0,2020-09-12,"('Mesut', 'Özil')",Fulham
1,6378,6,90,0,0,1,0,0,0,0,0,0,0,0,27,26.0,0.1,2.0,2.8,0,88657,5.0,2020-09-12,"('Federico', 'Fernández')",West Ham United
2,6394,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,3326,4.5,2020-09-12,"('Ciaran', 'Clark')",West Ham United
3,6410,7,90,0,0,1,0,0,0,0,0,0,3,0,26,27.0,0.0,0.0,2.7,0,13715,5.0,2020-09-12,"('Karl', 'Darlow')",West Ham United
4,6426,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,27245,5.0,2020-09-12,"('Martin', 'Dubravka')",West Ham United


In [3]:
ffmlDf.shape

(9685, 25)

In [4]:
# drop unneccessary columns
ffmlDf = ffmlDf.drop(columns=['Unnamed: 0','gameDate'])

# remove zerominsplayed - taken from 2_DataAnalysis
def RemoveZeroMinsPlayed(df):
    df = df.copy().where(df['minsPlayed'] != 0)
    df.dropna(axis=0, inplace=True)
    return df

ffmlDf = RemoveZeroMinsPlayed(ffmlDf)

ffmlDf.head()

,points,minsPlayed,goalsScored,assists,cleanSheets,goalsConceded,ownGoals,penSaved,penMissed,yelCards,redCards,saves,bonus,bonusPointSystem,influence,creativity,threat,ictIndex,netTransfers,selectedBy,costGBP,playerName,oppositionTeam
1,6.0,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,26.0,0.1,2.0,2.8,0.0,88657.0,5.0,"('Federico', 'Fernández')",West Ham United
3,7.0,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,26.0,27.0,0.0,0.0,2.7,0.0,13715.0,5.0,"('Karl', 'Darlow')",West Ham United
7,6.0,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,21.8,0.3,0.0,2.2,0.0,219489.0,4.5,"('Jamaal', 'Lascelles')",West Ham United
11,5.0,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,27.0,22.4,19.7,2.0,4.4,0.0,21964.0,4.5,"('Javier', 'Manquillo')",West Ham United
16,2.0,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,12.0,12.8,2.0,0.0,1.5,0.0,5941.0,5.0,"('Isaac', 'Hayden')",West Ham United


In [5]:
ffmlDf.shape

(4480, 23)

In [6]:
# separate data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(ffmlDf,
                                                   ffmlDf['points'],
                                                   test_size=0.2,
                                                   random_state=0)
# 80:20 split
# target is df['points']
X_train.shape, X_test.shape #everything is looking good so far

((3584, 23), (896, 23))

In [7]:
# check for missing variables

var_with_na = [
    var for var in ffmlDf.columns
    if X_train[var].isnull().sum() > 0
]
var_with_na
# if there were any vars with na we would, print the percentages
# for both the num and cat vars and fill in missing values

[]

In [8]:
# check the test set also has no-null values
[var for var in ffmlDf.columns if X_test[var].isnull().sum()]

[]

In [9]:
# rare labels need to be identified and removed from cat_vars
#identify cat_vars
cat_vars = [var for var in X_train.columns if X_train[var].dtype == 'O']
cat_vars

['playerName', 'oppositionTeam']

In [10]:
def FindFrequentLabels(df, var):
    df = df.copy()
    
    tmp = df.groupby(var)['points'].count() / len(df)
    
    print(tmp)
# 4480 is len
# 

for var in cat_vars:
    FindFrequentLabels(ffmlDf, var)

playerName
("N'Golo", 'Kanté')         0.003795
('Aaron', 'Connolly')       0.002455
('Aaron', 'Cresswell')      0.003795
('Aaron', 'Ramsdale')       0.003795
('Aaron', 'Wan-Bissaka')    0.003125
                              ...   
('Yerry', 'Mina')           0.003125
('Youri', 'Tielemans')      0.003795
('Yves', 'Bissouma')        0.003571
('Zack', 'Steffen')         0.000223
('Çaglar', 'Söyüncü')       0.001116
Name: points, Length: 460, dtype: float64
oppositionTeam
Arsenal                     0.051786
Aston Villa                 0.046205
Brighton and Hove Albion    0.052009
Burnley                     0.045759
Chelsea                     0.052009
Crystal Palace              0.052009
Everton                     0.049107
Fulham                      0.045536
Leeds                       0.052455
Leicester City              0.051786
Liverpool                   0.052009
Manchester City             0.045759
Manchester United           0.048214
Newcastle United            0.049107
Sheffie

In [11]:
# as we have sooo many players the Rare percentage will have to be made
# a lot lower than usual.
# opp team wont be affected by this

def FindFrequentLabels(df, var, rare_perc):
    df = df.copy()
    tmp = df.groupby(var)['points'].count() / len(df)
    return tmp[tmp > rare_perc].index

# Rare Labels Romoved and Replaced!!!

for var in cat_vars:
    frequent_list = FindFrequentLabels(ffmlDf, var, 0.001)
    # i want values of 0.001 so im being a little lenient i feel
    
    X_train[var] = np.where(X_train[var].isin(frequent_list), X_train[var], 'Rare')
    X_test[var] = np.where(X_test[var].isin(frequent_list), X_test[var], 'Rare')

# I am not sure wether this will add value tbh...

<ipython-input-11-0f217aa54426>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[var] = np.where(X_train[var].isin(frequent_list), X_train[var], 'Rare')
<ipython-input-11-0f217aa54426>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[var] = np.where(X_test[var].isin(frequent_list), X_test[var], 'Rare')
<ipython-input-11-0f217aa54426>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [12]:
print('X_train:',X_train['playerName'].nunique(), X_train.shape)
print('X_test:',X_test['playerName'].nunique(), X_test.shape)
# I am happy with this, train and test have equall columns
# but train has more unique values

X_train: 363 (3584, 23)
X_test: 329 (896, 23)


In [13]:
# WE WILL NOT BE TRANSFORMING NUMERICAL VARIABLES

In [14]:
# Encoding Categorical Variables
def ReplaceCategories(train, test, var, target):
    # order labels in a var from low to high
    ordered_labels = train.groupby(var)[target].mean().sort_values().index
    
    # create a dictionary of ordered labels to integer values
    ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}
    
    # use the dict to replace cat strings with integers
    train[var] = train[var].map(ordinal_label)
    test[var] = test[var].map(ordinal_label)

for var in cat_vars:
    ReplaceCategories(X_train, X_test, var, 'points')

<ipython-input-14-3baeee1eafdc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[var] = train[var].map(ordinal_label)
<ipython-input-14-3baeee1eafdc>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[var] = test[var].map(ordinal_label)


In [15]:
# check null values in train
[
    var for var in X_train.columns
    if X_train[var].isnull().sum() > 0
]

[]

In [16]:
# check null values in test
[
    var for var in X_test
    if X_test[var].isnull().sum() > 0
]

[]

In [17]:
# Feature Scaling!!!

# capture all vars in a list apart from TARGET
train_vars = [
    var for var in X_train.columns
    if var not in cat_vars and var != 'points' # could have used '!='...
]
train_vars, len(train_vars)

(['minsPlayed',
  'goalsScored',
  'assists',
  'cleanSheets',
  'goalsConceded',
  'ownGoals',
  'penSaved',
  'penMissed',
  'yelCards',
  'redCards',
  'saves',
  'bonus',
  'bonusPointSystem',
  'influence',
  'creativity',
  'threat',
  'ictIndex',
  'netTransfers',
  'selectedBy',
  'costGBP'],
 20)

In [18]:
# Create Scaler
scaler = MinMaxScaler()
# fit scaler to the train set - never the test set
scaler.fit(X_train[train_vars])
# transform the train and test set
X_train.to_csv('xtrain.csv', index=False)
X_test.to_csv('xtest.csv', index=False)